<a href="https://colab.research.google.com/github/Sheethaljoshi/TrainingSessionsForStudents/blob/main/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten
import numpy as np
import pandas as pd
from google.colab import drive
import os

In [2]:
# 1. Mount Google Drive
drive.mount('/content/drive')

# Define the folder path in Google Drive
drive_folder = "/content/drive/MyDrive/Datasets"
os.makedirs(drive_folder, exist_ok=True)  # Create the folder if it doesn't exist

# Paths for saving CSV files
train_csv_path = os.path.join(drive_folder, "mnist_train.csv")
test_csv_path = os.path.join(drive_folder, "mnist_test.csv")

# Check if the dataset already exists in Google Drive
if os.path.exists(train_csv_path) and os.path.exists(test_csv_path):
    print("Dataset already exists in Google Drive. Loading from CSV files...")
else:
    # 2. Download the MNIST dataset
    print("Downloading MNIST dataset...")
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Normalize the pixel values to [0, 1]
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Flatten the images (28x28 -> 784)
    x_train_flat = x_train.reshape(x_train.shape[0], -1)
    x_test_flat = x_test.reshape(x_test.shape[0], -1)

    y_train_onehot = tf.keras.utils.to_categorical(y_train, num_classes=10)
    y_test_onehot = tf.keras.utils.to_categorical(y_test, num_classes=10)

    # Combine features and labels into a single DataFrame
    train_df = pd.DataFrame(np.hstack([x_train_flat, y_train_onehot]))
    test_df = pd.DataFrame(np.hstack([x_test_flat, y_test_onehot]))

    # Save to CSV in Google Drive
    train_df.to_csv(train_csv_path, index=False, header=False)
    test_df.to_csv(test_csv_path, index=False, header=False)

    print(f"Training data saved to {train_csv_path}")
    print(f"Testing data saved to {test_csv_path}")


Mounted at /content/drive
Dataset already exists in Google Drive. Loading from CSV files...


In [3]:
print("Loading dataset from Google Drive...")
train_data = pd.read_csv(train_csv_path, header=None)
test_data = pd.read_csv(test_csv_path, header=None)

# Separate features and labels
x_train_csv = train_data.iloc[:, :-10].values  # First 784 columns are features
y_train_csv = train_data.iloc[:, -10:].values  # Last 10 columns are one-hot labels

x_test_csv = test_data.iloc[:, :-10].values
y_test_csv = test_data.iloc[:, -10:].values

Loading dataset from Google Drive...


In [4]:
# 3. Prepare the data (example using MNIST)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()  # Load MNIST dataset

# Preprocess the data (important!)
x_train = x_train.reshape(60000, 784).astype('float32') / 255.0  # Flatten and normalize pixel values
x_test = x_test.reshape(10000, 784).astype('float32') / 255.0

y_train = keras.utils.to_categorical(y_train, num_classes=10) # One-hot encode the labels
y_test = keras.utils.to_categorical(y_test, num_classes=10)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
# 4. Define the model
model = keras.Sequential([
    Flatten(input_shape=(28 * 28,)),  # Flatten input (784 features)
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# 5. Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 6. Train the model
model.fit(x_train_csv, y_train_csv, epochs=5, batch_size=32)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8769 - loss: 0.4267
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9638 - loss: 0.1220
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9755 - loss: 0.0814
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.9824 - loss: 0.0568
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9866 - loss: 0.0429


In [8]:
# 7. Evaluate on test data
loss, accuracy = model.evaluate(x_test_csv, y_test_csv, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

# 8. Make predictions (optional)
predictions = model.predict(x_test_csv)
predicted_labels = tf.argmax(predictions, axis=1)

# Display first 10 predictions
print(predicted_labels[:10])

Test loss: 0.07255573570728302
Test accuracy: 0.9779999852180481
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
tf.Tensor([7 2 1 0 4 1 4 9 5 9], shape=(10,), dtype=int64)
